In [20]:
import os
import logging
from clickhouse_driver import Client
from dotenv import load_dotenv
from datetime import datetime, timedelta, date

# Загрузка переменных окружения из файла .env
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,  # Уровень логирования INFO для менее подробного вывода
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("step_iv.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Параметры подключения к ClickHouse
CLICKHOUSE_HOST = os.getenv('CLICKHOUSE_HOST', '10.95.19.132')
CLICKHOUSE_USER = os.getenv('CLICKHOUSE_USER', 'default')
CLICKHOUSE_PASSWORD = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
CLICKHOUSE_DB = os.getenv('CLICKHOUSE_DB', 'default')

# Создание клиента ClickHouse
client = Client(
    host=CLICKHOUSE_HOST,
    user=CLICKHOUSE_USER,
    password=CLICKHOUSE_PASSWORD,
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)

def get_initial_date():
    """
    Получает самую раннюю дату из таблицы OlapCube_VNV.
    """
    try:
        query = f"""
        SELECT MIN(Dates) AS initial_date
        FROM {CLICKHOUSE_DB}.OlapCube_VNV
        """
        result = client.execute(query)
        initial_date = result[0][0]
        logger.info(f"Начальная дата в OlapCube_VNV: {initial_date}")
        return initial_date
    except Exception as e:
        logger.error(f"Ошибка при получении начальной даты: {e}", exc_info=True)
        return None

def fetch_previous_and_target_data(target_date):
    """
    Извлекает данные для target_date -1 и target_date из OlapCube_VNV.
    Также извлекает текущее значение Status_P и Status для target_date.
    
    :param target_date: Дата, для которой выполняется обновление (datetime.date)
    :return: Кортеж из двух словарей (prev_dict, target_dict)
    """
    try:
        previous_date = target_date - timedelta(days=1)
        logger.info(f"Обработка предыдущей даты: {previous_date}")
        logger.info(f"Обработка target_date: {target_date}")

        # Извлечение данных за предыдущую дату
        query_prev = f"""
        SELECT 
            serialno, Status_P, sne, ppr, ll, oh, BR, daily_flight_hours, repair_days, RepairTime, ac_typ
        FROM {CLICKHOUSE_DB}.OlapCube_VNV
        WHERE Dates = '{previous_date}'
        """
        prev_data = client.execute(query_prev)
        logger.info(f"Извлечено {len(prev_data)} записей за предыдущую дату {previous_date}.")

        # Извлечение данных за target_date, включая текущее значение Status_P и Status
        query_target = f"""
        SELECT 
            serialno, Status_P, sne, ppr, ll, oh, BR, daily_flight_hours, repair_days, RepairTime, ac_typ, Status
        FROM {CLICKHOUSE_DB}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        """
        target_data = client.execute(query_target)
        logger.info(f"Извлечено {len(target_data)} записей за target_date {target_date}.")

        # Преобразование данных в словари для быстрого доступа по serialno
        prev_dict = {row[0]: row for row in prev_data}
        target_dict = {row[0]: row for row in target_data}

        return prev_dict, target_dict
    except Exception as e:
        logger.error(f"Ошибка при извлечении данных: {e}", exc_info=True)
        return None, None

def update_counters(prev_dict, target_dict, target_date):
    """
    Обновляет счетчики sne, ppr и repair_days в таблице OlapCube_VNV на target_date согласно заданной логике.
    
    :param prev_dict: Словарь с данными за предыдущую дату, ключ = serialno
    :param target_dict: Словарь с данными за target_date, ключ = serialno
    :param target_date: Дата, для которой выполняется обновление (datetime.date)
    """
    counter_updates = []
    unresolved = []
    total_updates = 0  # Счетчик успешных обновлений

    for serialno, target_row in target_dict.items():
        current_status = target_row[11]  # Поле Status (индекс 11)
        current_daily_flight_hours = target_row[7]  # Поле daily_flight_hours (индекс 7)

        prev_row = prev_dict.get(serialno, None)
        if not prev_row:
            unresolved.append(serialno)
            continue

        prev_status_p = prev_row[1]  # Status_P на предыдущую дату (индекс 1)
        prev_sne = prev_row[2]       # sne на предыдущую дату (индекс 2)
        prev_ppr = prev_row[3]       # ppr на предыдущую дату (индекс 3)
        prev_repair_days = prev_row[8]  # repair_days на предыдущую дату (индекс 8)

        # Приведение типов
        try:
            prev_sne = float(prev_sne) if prev_sne is not None else 0.0
            prev_ppr = float(prev_ppr) if prev_ppr is not None else 0.0
            # repair_days может быть NULL, оставляем как есть
            daily_flight_hours = float(current_daily_flight_hours) if current_daily_flight_hours is not None else 0.0
        except ValueError:
            unresolved.append(serialno)
            continue

        # Инициализация новых значений
        new_sne = prev_sne
        new_ppr = prev_ppr
        new_repair_days = prev_repair_days  # По умолчанию

        if current_status == "Эксплуатация":
            new_sne += daily_flight_hours
            new_ppr += daily_flight_hours

        elif current_status == "Исправен":
            if prev_status_p == "Ремонт":
                new_ppr = 0.0
                new_repair_days = None  # Устанавливаем в NULL
            else:
                new_ppr = prev_ppr

        elif current_status == "Ремонт":
            if prev_status_p == "Эксплуатация":
                new_repair_days = 1.0
            elif prev_repair_days is not None:
                new_repair_days += 1.0
            else:
                new_repair_days = 1.0  # Если предыдущие repair_days были NULL

        elif current_status in ["Хранение", "Неактивно"]:
            # sne и ppr остаются без изменений
            pass

        else:
            unresolved.append(serialno)
            continue

        # Округление до двух знаков после запятой
        new_sne = round(new_sne, 2)
        new_ppr = round(new_ppr, 2)
        if new_repair_days is not None:
            new_repair_days = round(new_repair_days, 2)

        # Добавляем обновленные значения в список
        counter_updates.append({
            'serialno': serialno,
            'sne': new_sne,
            'ppr': new_ppr,
            'repair_days': new_repair_days,
            'date': target_date
        })

    # Выполнение обновлений
    try:
        for update in counter_updates:
            serialno = update['serialno']
            sne = update['sne']
            ppr = update['ppr']
            repair_days = update['repair_days']
            date_val = update['date']

            # Формирование SQL-запроса с учетом возможности NULL для repair_days
            if repair_days is None:
                update_query = f"""
                ALTER TABLE {CLICKHOUSE_DB}.OlapCube_VNV 
                UPDATE sne = %(sne)s, ppr = %(ppr)s, repair_days = NULL 
                WHERE serialno = %(serialno)s AND Dates = %(date)s
                """
                params = {
                    'sne': sne,
                    'ppr': ppr,
                    'serialno': serialno,
                    'date': date_val
                }
            else:
                update_query = f"""
                ALTER TABLE {CLICKHOUSE_DB}.OlapCube_VNV 
                UPDATE sne = %(sne)s, ppr = %(ppr)s, repair_days = %(repair_days)s 
                WHERE serialno = %(serialno)s AND Dates = %(date)s
                """
                params = {
                    'sne': sne,
                    'ppr': ppr,
                    'repair_days': repair_days,
                    'serialno': serialno,
                    'date': date_val
                }

            try:
                client.execute(update_query, params)
                total_updates += 1
            except Exception as e_inner:
                unresolved.append(serialno)
                continue  # Продолжаем обработку остальных записей

        # Логирование общего количества изменений
        logger.info(f"Обновлено {total_updates} записей счетчиков sne, ppr и repair_days для даты {target_date}.")

        if unresolved:
            logger.warning(f"Не удалось обновить счетчики для следующих serialno: {', '.join(unresolved)}")
    except Exception as e:
        logger.error(f"Ошибка при обновлении счетчиков: {e}", exc_info=True)

def main():
    """
    Главная функция, выполняющая шаг IV алгоритма.
    """
    # Шаг 1: Получение начальной даты
    initial_date = get_initial_date()
    if not initial_date:
        logger.error("Не удалось определить начальную дату. Завершаем выполнение.")
        return

    # Преобразование initial_date в объект date, если это строка или datetime
    if isinstance(initial_date, str):
        try:
            initial_date = datetime.strptime(initial_date, "%Y-%m-%d").date()
        except ValueError as ve:
            logger.error(f"Неверный формат начальной даты: {initial_date}. Ошибка: {ve}", exc_info=True)
            return
    elif isinstance(initial_date, datetime):
        initial_date = initial_date.date()

    # Определение target_date
    target_date = initial_date + timedelta(days=1)
    logger.info(f"Определен target_date: {target_date}")

    # Извлечение данных для предыдущей и target_date
    prev_dict, target_dict = fetch_previous_and_target_data(target_date)
    if not prev_dict or not target_dict:
        logger.error("Не удалось извлечь необходимые данные. Завершаем выполнение.")
        return

    # Шаг IV: Обновление счетчиков sne, ppr и repair_days
    update_counters(prev_dict, target_dict, target_date)

if __name__ == "__main__":
    main()


2024-12-04 06:07:08,193 - INFO - Начальная дата в OlapCube_VNV: 2024-11-25
2024-12-04 06:07:08,194 - INFO - Определен target_date: 2024-11-26
2024-12-04 06:07:08,195 - INFO - Обработка предыдущей даты: 2024-11-25
2024-12-04 06:07:08,196 - INFO - Обработка target_date: 2024-11-26
2024-12-04 06:07:08,324 - INFO - Извлечено 420 записей за предыдущую дату 2024-11-25.
2024-12-04 06:07:08,450 - INFO - Извлечено 420 записей за target_date 2024-11-26.
2024-12-04 06:07:13,359 - INFO - Обновлено 420 записей счетчиков sne, ppr и repair_days для даты 2024-11-26.
